<a href="https://colab.research.google.com/github/chakriga/Hadoop_Assignments/blob/main/Pyspark_Assignment(Pb1%20%26%20Pb2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#install Apache Spark 3.0.1 with Hadoop 2.7 from here.
!wget https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz

# Now, we just need to unzip that folder.
!tar -xvzf spark-3.0.0-bin-hadoop2.7.tgz
!pip install findspark


import os
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"
import findspark
findspark.init()

--2023-12-13 13:44:42--  https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 220272364 (210M) [application/x-gzip]
Saving to: ‘spark-3.0.0-bin-hadoop2.7.tgz’

spark-3.0.0-bin-had 100%[===================>] 210.07M   977KB/s    in 4m 51s  

2023-12-13 13:49:34 (739 KB/s) - ‘spark-3.0.0-bin-hadoop2.7.tgz’ saved [220272364/220272364]

spark-3.0.0-bin-hadoop2.7/
spark-3.0.0-bin-hadoop2.7/NOTICE
spark-3.0.0-bin-hadoop2.7/kubernetes/
spark-3.0.0-bin-hadoop2.7/kubernetes/tests/
spark-3.0.0-bin-hadoop2.7/kubernetes/tests/worker_memory_check.py
spark-3.0.0-bin-hadoop2.7/kubernetes/tests/py_container_checks.py
spark-3.0.0-bin-hadoop2.7/kubernetes/tests/pyfiles.py
spark-3.0.0-bin-hadoop2.7/kubernetes/dockerfiles/
spark-3.0.0-bin-hadoop2.7/ku

#1.Load the covid 19 data in to a spark dataframe (country_wise_latest.csv) with the correct schema definition


In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DecimalType
#create sparksession
spark = SparkSession.builder.master("local").appName("DemoApp").getOrCreate()
print(type(spark))

doma=StructType([
    StructField("Country/Region",StringType(),True),
    StructField("Confirmed", IntegerType(),True),
    StructField("Deaths", IntegerType(),True),
    StructField("Recovered", IntegerType(),True),
    StructField("Active", IntegerType(),True),
    StructField("New cases", IntegerType(),True),
    StructField("New deaths", IntegerType(),True),
    StructField("New recovered", IntegerType(),True),
    StructField("Deaths / 100 Cases",DecimalType(),True),
    StructField("Recovered / 100 Cases",DecimalType(),True),
    StructField("Deaths / 100 Recovered",DecimalType(),True),
    StructField("Confirmed last week",IntegerType(),True),
    StructField("1 week change",IntegerType(),True),
    StructField("1 week % increase",DecimalType(),True),
    StructField("WHO Region",StringType(),True)
])


df = spark.read.csv(path='/content/covid19.csv',header=True,schema=doma)
df.show()

<class 'pyspark.sql.session.SparkSession'>
+-------------------+---------+------+---------+------+---------+----------+-------------+------------------+---------------------+----------------------+-------------------+-------------+-----------------+--------------------+
|     Country/Region|Confirmed|Deaths|Recovered|Active|New cases|New deaths|New recovered|Deaths / 100 Cases|Recovered / 100 Cases|Deaths / 100 Recovered|Confirmed last week|1 week change|1 week % increase|          WHO Region|
+-------------------+---------+------+---------+------+---------+----------+-------------+------------------+---------------------+----------------------+-------------------+-------------+-----------------+--------------------+
|        Afghanistan|    36263|  1269|    25198|  9796|      106|        10|           18|                 4|                   69|                     5|              35526|          737|                2|Eastern Mediterra...|
|            Albania|     4880|   144|     27

#2. The are some column names which are long, contains specialcharacters, spaces etc. Rename all such column names accordingly.Example Country/Region → country New cases → New_cases etc

In [22]:
mapping={
    "Country/Region":"Country",
    "New cases":"New_cases",
    "New deaths":"New_deaths",
    "New recovered":"New_recovered",
    "Deaths / 100 Cases":"Death_rate",
    "Recovered / 100 Cases":"Recovery_rate",
    "Deaths / 100 Recovered":"death_to_recovery_ratio",
    "Confirmed last week":"Confirmed_last_week",
    "1 week change":"1_week_change",
    "1 week % increase":"1_week_Percentage_increase",
    "WHO Region":"WHO_Region"
}

for ocol,ncol in mapping.items():
  df=df.withColumnRenamed(ocol,ncol)
df.show()



+-------------------+---------+------+---------+------+---------+----------+-------------+----------+-------------+-----------------------+-------------------+-------------+--------------------------+--------------------+
|            Country|Confirmed|Deaths|Recovered|Active|New_cases|New_deaths|New_recovered|Death_rate|Recovery_rate|death_to_recovery_ratio|Confirmed_last_week|1_week_change|1_week_Percentage_increase|          WHO_Region|
+-------------------+---------+------+---------+------+---------+----------+-------------+----------+-------------+-----------------------+-------------------+-------------+--------------------------+--------------------+
|        Afghanistan|    36263|  1269|    25198|  9796|      106|        10|           18|         4|           69|                      5|              35526|          737|                         2|Eastern Mediterra...|
|            Albania|     4880|   144|     2745|  1991|      117|         6|           63|         3|           

#3. Count and check if there any null values in any of the columns

In [23]:
for col in df.columns:
    null_count = df.filter(df[col].isNull()).count()
    print(f"Number of null values in {col}: {null_count}")


Number of null values in Country: 0
Number of null values in Confirmed: 0
Number of null values in Deaths: 0
Number of null values in Recovered: 0
Number of null values in Active: 0
Number of null values in New_cases: 0
Number of null values in New_deaths: 0
Number of null values in New_recovered: 0
Number of null values in Death_rate: 0
Number of null values in Recovery_rate: 0
Number of null values in death_to_recovery_ratio: 5
Number of null values in Confirmed_last_week: 0
Number of null values in 1_week_change: 0
Number of null values in 1_week_Percentage_increase: 0
Number of null values in WHO_Region: 0


#4. What are the top 10 countries under the WHO region with covid 19 Confirmed cases
from pyspark.sql import functions as F

In [24]:
from pyspark.sql import functions as F
who_region_col = 'WHO_Region'
confirmed_col = 'Confirmed'

df.createOrReplaceTempView("covid_data")
top_ten_sql = f"""
    SELECT
        Country,
        {confirmed_col},
        {who_region_col},
        ROW_NUMBER() OVER (PARTITION BY {who_region_col} ORDER BY {confirmed_col} DESC) AS rank
    FROM covid_data
"""

ranked_countries_df = spark.sql(top_ten_sql)
top10_per_region = ranked_countries_df.filter(F.col('rank') <= 10)
top10_per_region.show()



+--------------+---------+---------------+----+
|       Country|Confirmed|     WHO_Region|rank|
+--------------+---------+---------------+----+
|        Russia|   816680|         Europe|   1|
|United Kingdom|   301708|         Europe|   2|
|         Spain|   272421|         Europe|   3|
|         Italy|   246286|         Europe|   4|
|        Turkey|   227019|         Europe|   5|
|        France|   220352|         Europe|   6|
|       Germany|   207112|         Europe|   7|
|    Kazakhstan|    84648|         Europe|   8|
|        Sweden|    79395|         Europe|   9|
|       Belarus|    67251|         Europe|  10|
|         China|    86783|Western Pacific|   1|
|   Philippines|    82040|Western Pacific|   2|
|     Singapore|    50838|Western Pacific|   3|
|         Japan|    31142|Western Pacific|   4|
|     Australia|    15303|Western Pacific|   5|
|   South Korea|    14203|Western Pacific|   6|
|      Malaysia|     8904|Western Pacific|   7|
|   New Zealand|     1557|Western Pacifi

#5. What are the bottom 10 countries under the WHO region with covid 19 Confirmed cases

In [25]:
bottom_ten_sql= f"""
    SELECT
        Country,
        {confirmed_col},
        {who_region_col},
        ROW_NUMBER() OVER (PARTITION BY {who_region_col} ORDER BY {confirmed_col} ASC) AS rank
    FROM covid_data
"""

bottom_df=spark.sql(bottom_ten_sql)
bottom10_per_region = bottom_df.filter(F.col('rank') <= 10)
bottom10_per_region.show()

+----------------+---------+---------------+----+
|         Country|Confirmed|     WHO_Region|rank|
+----------------+---------+---------------+----+
|        Holy See|       12|         Europe|   1|
|       Greenland|       14|         Europe|   2|
|   Liechtenstein|       86|         Europe|   3|
|          Monaco|      116|         Europe|   4|
|      San Marino|      699|         Europe|   5|
|           Malta|      701|         Europe|   6|
|         Andorra|      907|         Europe|   7|
|          Cyprus|     1060|         Europe|   8|
|         Georgia|     1137|         Europe|   9|
|          Latvia|     1219|         Europe|  10|
|            Laos|       20|Western Pacific|   1|
|            Fiji|       27|Western Pacific|   2|
|Papua New Guinea|       62|Western Pacific|   3|
|          Brunei|      141|Western Pacific|   4|
|        Cambodia|      226|Western Pacific|   5|
|        Mongolia|      289|Western Pacific|   6|
|         Vietnam|      431|Western Pacific|   7|


#6. What are the total number of countries/ total no. of WHO regions and also list the various WHO regions

In [26]:
total_countries = df.select('Country').distinct().count()
print(f"Total number of countries: {total_countries}")


total_who_regions = df.select('WHO_Region').distinct().count()
print(f"Total number of WHO regions: {total_who_regions}")

who_regions_list = [row['WHO_Region'] for row in df.select('WHO_Region').distinct().na.drop().collect()]
print(f"List of WHO regions: {who_regions_list}")





Total number of countries: 187
Total number of WHO regions: 6
List of WHO regions: ['Europe', 'Western Pacific', 'Africa', 'Eastern Mediterranean', 'Americas', 'South-East Asia']


pb2

In [29]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F


spark = SparkSession.builder.appName("CrimeDataAnalysis").getOrCreate()


crime_data_df = spark.read.csv("/content/crime2013.csv", header=True,sep=',', inferSchema=True)

crime_data_df.show()

crime_data_df.printSchema()
total_offenses = crime_data_df.select('OFFENSE').distinct().count()
print(f"Total number of offenses in 2013: {total_offenses}")


+--------+--------------------+--------+-----------+------+--------------------+--------------------+----------------+----------------+----+---+--------+---+-------------------+---------------------------+-----------+------------+---------------+--------------------+--------------------+
|     CCN|      REPORTDATETIME|   SHIFT|    OFFENSE|METHOD|    LASTMODIFIEDDATE|    BLOCKSITEADDRESS|     BLOCKXCOORD|     BLOCKYCOORD|WARD|ANC|DISTRICT|PSA|NEIGHBORHOODCLUSTER|BUSINESSIMPROVEMENTDISTRICT|BLOCK_GROUP|CENSUS_TRACT|VOTING_PRECINCT|          START_DATE|            END_DATE|
+--------+--------------------+--------+-----------+------+--------------------+--------------------+----------------+----------------+----+---+--------+---+-------------------+---------------------------+-----------+------------+---------------+--------------------+--------------------+
| 4104147|4/16/2013 12:00:0...|MIDNIGHT|   HOMICIDE| KNIFE|5/15/2015 8:33:04 AM|1500 - 1599 BLOCK...|        398943.0|        133729.

# Task 8: How many crimes were committed in each offense?

In [30]:
crimes_per_offense = crime_data_df.groupBy('OFFENSE').count().orderBy(F.desc('count'))
crimes_per_offense.show()


+--------------------+-----+
|             OFFENSE|count|
+--------------------+-----+
|         THEFT/OTHER|11269|
|        THEFT F/AUTO| 9185|
|             ROBBERY| 3967|
|            BURGLARY| 3296|
| MOTOR VEHICLE THEFT| 2611|
|ASSAULT W/DANGERO...| 2253|
|           SEX ABUSE|  293|
|            HOMICIDE|  102|
|               ARSON|   33|
|                null|    1|
+--------------------+-----+



# Task 9: How many different methods were used (And their count) in offense “Homicide”

In [31]:
homicide_methods_count = (
    crime_data_df.filter(crime_data_df['Offense'] == "HOMICIDE")
                .groupBy('Method')
                .agg(F.count('Method').alias('MethodCount'))
                .orderBy(F.desc('MethodCount'))
)
homicide_methods_count.show(truncate=False)


+------+-----------+
|Method|MethodCount|
+------+-----------+
|GUN   |80         |
|OTHERS|11         |
|KNIFE |11         |
+------+-----------+



# Task 10: Which shift had the maximum crimes?

In [32]:
max_crime_shift = (
    crime_data_df.groupBy('Shift')
                .count()
                .orderBy(F.desc('count'))
                .select('Shift')
                .first()
                .Shift
)
print(f"The shift with the maximum crimes is: {max_crime_shift}")

# Stop the Spark session when done
spark.stop()


The shift with the maximum crimes is: EVENING
